In [11]:
import mackinac

In [12]:
mackinac.modelseed.ms_client.url = 'https://p3.theseed.org/services/ProbModelSEED/'
mackinac.workspace.ws_client.url = 'https://p3.theseed.org/services/Workspace'
mackinac.genome.patric_url = 'https://www.patricbrc.org/api/'

In [13]:
mackinac.get_token("fcomnozz")

patric password: ········


'fcomnozz@patricbrc.org'

In [14]:
cobra_model = mackinac.create_cobra_model_from_modelseed_model(model_id="226186.12")

In [15]:
cobra_model.reactions.rxn00011_c

Reaction identifier,rxn00011_c
Name,pyruvate:thiamin diphosphate acetaldehydetransferase (decarboxylating)
Memory address,0x07fd2d538d9a0
Stoichiometry,cpd00020_c + cpd00056_c + cpd00067_c --> cpd00011_c + cpd03049_c Pyruvate_c + TPP_c + H+_c --> CO2_c + 2-Hydroxyethyl-ThPP_c
GPR,( 226186.12.peg.2137 and 226186.12.peg.2138 )
Lower bound,0.0
Upper bound,1000.0


In [16]:
# this function can be useful for extracting the name of organism
mackinac.list_modelseed_models(print_output=True)

Model /fcomnozz@patricbrc.org/modelseed/226186.12 for organism Bacteroides thetaiotaomicron VPI-5482 with 967 reactions and 1142 metabolites
